# newer crf trained with kfolded data

In [5]:
#Data Analysis
import pandas as pd
import numpy as np

#Data Visualisation
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(color_codes=True)
sns.set(font_scale=1)
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

from copy import deepcopy
from collections import defaultdict

#Modeling
import fasttext
from sklearn.model_selection import cross_val_predict, cross_val_score, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn_crfsuite import CRF, scorers, metrics
from sklearn_crfsuite.metrics import flat_classification_report
from sklearn.metrics import classification_report, make_scorer, precision_recall_fscore_support
import scipy.stats
# import eli5

from nervaluate import Evaluator

In [7]:
path = "C:\\Users\\farih\\Downloads\\"
# indoNLUmodel = "..\\fasttext.4B.id.300.epoch5.uncased.bin"
model = fasttext.load_model(indoNLUmodel)

In [ ]:
train_df = pd.read_csv(".\\..\\Dataset\\StratifiedKFold\\trainval.csv")
test_df = pd.read_csv(".\\..\\Dataset\\StratifiedKFold\\test.csv")

In [2]:
train_df.head(1)

NameError: name 'train_df' is not defined

In [5]:
def getsentence(df):
    def _zip(a,b):
            temp_list= list()
            for i in range(len(a)):
                temp_list.append((a[i],b[i]))
            return temp_list
    sentences = list()
    for idx, row in df.iterrows():
        zipped_sentence = tuple(_zip(eval(row.token),eval(row.ner)))
        sentences.append(zipped_sentence)
    return sentences

In [6]:
sentences_train = getsentence(train_df)
sentences_test = getsentence(test_df)
print(sentences_train[:1])
print(sentences_test[:1])


[(('@ridwankamil', 'O'), ('@KickAndyShow', 'O'), ('@Metro_TV', 'O'), ('Upaya', 'O'), ('untuk', 'O'), ('mnaikan', 'O'), ('nilai', 'O'), ('jual', 'O'), ('yg', 'O'), ('bgus,dngn', 'O'), ('cara', 'O'), ('d', 'O'), ('desain', 'O'), ('pak', 'O'), ('gubernur', 'B-PER'), ('akan', 'O'), ('menjadikn', 'O'), ('produk', 'O'), ('umkm', 'O'), ('d', 'O'), ('mintai', 'O'), ('krena', 'O'), ('yg', 'O'), ('desain', 'O'), ('org', 'O'), ('nomer', 'O'), ('satu', 'O'), ('di', 'O'), ('jabar', 'B-LOC'))]
[(('@GyuuPotter', 'O'), ('iyhh', 'O'), (',', 'O'), ('soalnya', 'O'), ('mwu', 'O'), ('kerjain', 'O'), ('tugas', 'O'), ('jg��', 'O'))]


In [7]:
# words = list(train_df["token"]))
# n_words = len(words)
# print(n_words)

In [10]:

# Feature set
def word2features(sent, i):
    word = sent[i][0]
    
    features = {
        'bias': 1.0,
        'word.lower()': word.lower() if type(word) == str else word,
        'word[-3:]': word[-3:] if type(word) == str else word,
        'word[-2:]': word[-2:] if type(word) == str else word,
        'word[-1:]': word[-2:] if type(word) == str else word,
        'word[:3]': word[:3] if type(word) == str else word,
        'word[:2]': word[:2] if type(word) == str else word,
        'word[:1]': word[:2] if type(word) == str else word,
        'word.isupper()': word.isupper() if type(word) == str else word,
        'word.istitle()': word.istitle() if type(word) == str else word,
        'word.isdigit()': word.isdigit() if type(word) == str else word,
    }
    if i > 0:
        word1 = sent[i-1][0]
        features.update({
            '-1:word.lower()': word1.lower() if type(word1) == str else word1,
            '-1:word.istitle()': word1.istitle() if type(word1) == str else word1,
            '-1:word.isupper()': word1.isupper() if type(word1) == str else word1,
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        features.update({
            '+1:word.lower()': word1.lower() if type(word1) == str else word1,
            '+1:word.istitle()': word1.istitle() if type(word1) == str else word1,
            '+1:word.isupper()': word1.isupper() if type(word1) == str else word1,
        })
    else:
        features['EOS'] = True

    #src: https://stackoverflow.com/questions/58736548/how-to-use-word-embedding-as-features-for-crf-sklearn-crfsuite-mod   el-training
    we=model.get_word_vector(features['word.lower()'])
    for iv,value in enumerate(we):
        features['v{}'.format(iv)]=value

    return features

def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, label in sent]

In [11]:
word2features([['@ridwankamil']],0)

{'bias': 1.0,
 'word.lower()': '@ridwankamil',
 'word[-3:]': 'mil',
 'word[-2:]': 'il',
 'word[-1:]': 'il',
 'word[:3]': '@ri',
 'word[:2]': '@r',
 'word[:1]': '@r',
 'word.isupper()': False,
 'word.istitle()': False,
 'word.isdigit()': False,
 'BOS': True,
 'EOS': True,
 'v0': -0.14921397,
 'v1': -0.34261304,
 'v2': -0.14893499,
 'v3': -0.23975867,
 'v4': -0.09407631,
 'v5': -0.02673461,
 'v6': 0.011205231,
 'v7': 0.09165662,
 'v8': -0.12411242,
 'v9': -0.43121994,
 'v10': -0.1770174,
 'v11': -0.6441361,
 'v12': -0.099531695,
 'v13': -0.11209641,
 'v14': -0.182792,
 'v15': 0.53043723,
 'v16': 0.10754142,
 'v17': 0.5619152,
 'v18': -0.20820382,
 'v19': -0.32403356,
 'v20': 0.12107727,
 'v21': -0.29408154,
 'v22': -0.24585378,
 'v23': 0.026907355,
 'v24': -0.6294125,
 'v25': -0.059103522,
 'v26': -0.008576035,
 'v27': -0.18828441,
 'v28': -0.39511928,
 'v29': 0.5678898,
 'v30': 0.22642602,
 'v31': -0.22020207,
 'v32': 0.4680126,
 'v33': 0.32993266,
 'v34': -0.39405876,
 'v35': -0.110538

## Feature Extraction

In [9]:
%%time
#Creating the train and test set
X_train = [sent2features(s) for s in sentences_train]
X_test = [sent2features(s) for s in sentences_test]
y_train = [sent2labels(s) for s in sentences_train]
y_test = [sent2labels(s) for s in sentences_test]

Wall time: 19.6 s


In [10]:
# group B and I results
# #First we select all the tags that are relevant for us i.e. remove the 'O' tag from the list. 

# labels = list(crf2.classes_)
# labels = list(train_df['labels'].unique())
# labels = list(filter(lambda a: a != 'O', labels))
# sorted_labels = sorted(
#     labels,
#     key=lambda name: (name[1:], name[0])
# )
sorted_labels=['B-EV', 'I-EV', 'B-LOC', 'I-LOC', 'B-ORG', 'I-ORG', 'B-PER', 'I-PER', 'B-PROD', 'I-PROD', 'B-WA', 'I-WA']
print(sorted_labels)

['B-EV', 'I-EV', 'B-LOC', 'I-LOC', 'B-ORG', 'I-ORG', 'B-PER', 'I-PER', 'B-PROD', 'I-PROD', 'B-WA', 'I-WA']


## Training

In [11]:
%%time
#Now we will create the Randomized CV search model wherein we will use a modified F1 scorer model considering only the relevant labels
# define fixed parameters and parameters to search
crf = CRF(
    algorithm='lbfgs',
    max_iterations=100,
    all_possible_transitions=True
)   
params_space = {
    'c1': scipy.stats.expon(scale=0.5),
    'c2': scipy.stats.expon(scale=0.05),
}

# use the same metric for evaluation
f1_scorer = make_scorer(metrics.flat_f1_score,
                        average='weighted', labels=sorted_labels)

# search
rs = RandomizedSearchCV(crf, params_space,
                        cv=3,
                        verbose=1,
                        n_jobs=8, #-1
                        n_iter=50 ,#50,
                        scoring=f1_scorer)
rs.fit(X_train, y_train)

Fitting 3 folds for each of 50 candidates, totalling 150 fits


d:\ner-idn-tweet\env\lib\site-packages\sklearn\base.py:209: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  warnings.warn('From version 0.24, get_params will raise an '
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
d:\ner-idn-tweet\env\lib\site-packages\joblib\externals\loky\process_executor.py:702: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed: 59.2min
[Parallel(n_jobs=8)]: Done 150 out of 150 | elapsed: 252.2min finished


Wall time: 4h 14min 12s


RandomizedSearchCV(cv=3,
                   estimator=CRF(algorithm='lbfgs',
                                 all_possible_transitions=True,
                                 keep_tempfiles=None, max_iterations=100),
                   n_iter=50, n_jobs=8,
                   param_distributions={'c1': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001F81F2DBE80>,
                                        'c2': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001F81F33B490>},
                   scoring=make_scorer(flat_f1_score, average=weighted, labels=['B-EV', 'I-EV', 'B-LOC', 'I-LOC', 'B-ORG', 'I-ORG', 'B-PER', 'I-PER', 'B-PROD', 'I-PROD', 'B-WA', 'I-WA']),
                   verbose=1)

In [12]:
#Lets check the best estimated parameters and CV score
print('Best parameters:', rs.best_params_)
print('Best CV score:', rs.best_score_)
print('Model size: {:0.2f}M'.format(rs.best_estimator_.size_ / 1000000))

Best parameters: {'c1': 0.21444029943694473, 'c2': 0.16763911560517203}
Best CV score: 0.6758272249722509
Model size: 2.16M


## Performance per label type per token

In [14]:
#Now we create the model again using the best estimators
best_crf = rs.best_estimator_
y_pred = best_crf.predict(X_test)
print(metrics.flat_classification_report(
    y_test, y_pred, labels=sorted_labels, digits=3
    ))
# print(precision_recall_fscore_support(y_test, y_pred, labels=sorted_labels, digits=3))

              precision    recall  f1-score   support

        B-EV      0.616     0.304     0.407       148
        I-EV      0.537     0.367     0.436       199
       B-LOC      0.771     0.787     0.779       437
       I-LOC      0.686     0.602     0.641       196
       B-ORG      0.659     0.452     0.537       389
       I-ORG      0.554     0.482     0.515       193
       B-PER      0.904     0.742     0.815      1148
       I-PER      0.888     0.864     0.876       603
      B-PROD      0.876     0.827     0.851       676
      I-PROD      0.741     0.576     0.648       243
        B-WA      0.333     0.036     0.065        28
        I-WA      0.250     0.064     0.102        47

   micro avg      0.805     0.679     0.737      4307
   macro avg      0.651     0.509     0.556      4307
weighted avg      0.789     0.679     0.725      4307



In [ ]:
best_crf.fit(X_train,y_train)

CRF(algorithm='lbfgs', all_possible_transitions=True, c1=0.1292091096846934,
    c2=0.11790564639103854, keep_tempfiles=None, max_iterations=100)

In [ ]:
# eli5.show_weights(crf3, top=30)

## Performance over full named-entity

In [15]:
ner_tag = ['LOC', 'PER', 'ORG', 'PROD', 'EV' ,'WA']

## Result over all messages

In [16]:
evaluator = Evaluator(y_test, y_pred, tags=['LOC', 'PER', 'ORG', 'PROD', 'WA', 'EV'], loader="list")

In [17]:
results, results_per_tag = evaluator.evaluate()

In [18]:
results['partial']

{'correct': 1991,
 'incorrect': 0,
 'partial': 211,
 'missed': 655,
 'spurious': 181,
 'possible': 2857,
 'actual': 2383,
 'precision': 0.8797733948804028,
 'recall': 0.7338116905845292,
 'f1': 0.8001908396946564}

In [19]:
results['exact']

{'correct': 1991,
 'incorrect': 211,
 'partial': 0,
 'missed': 655,
 'spurious': 181,
 'possible': 2857,
 'actual': 2383,
 'precision': 0.8355014687368862,
 'recall': 0.6968848442422121,
 'f1': 0.7599236641221373}

In [20]:
with open("./results_1_batch.txt", "w") as f:
    str_results = repr(results)
    f.write(str_results)

In [21]:
with open("./results_per_tag_1_batch.txt", "w") as f:
    str_results_per_tag = repr(results_per_tag)
    f.write(str_results_per_tag)

In [22]:
import pickle

# save
with open('CRF_WE_model.pkl','wb') as f:
    pickle.dump(best_crf,f)



In [24]:
# load
with open('CRF_WE_model.pkl', 'rb') as f:
    crf2 = pickle.load(f)

In [25]:

y_pred = crf2.predict(X_test)
print(metrics.flat_classification_report(
    y_test, y_pred, labels=sorted_labels, digits=3
    ))
# print(precision_recall_fscore_support(y_test, y_pred, labels=sorted_labels, digits=3))

d:\ner-idn-tweet\env\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass labels=['B-EV', 'I-EV', 'B-LOC', 'I-LOC', 'B-ORG', 'I-ORG', 'B-PER', 'I-PER', 'B-PROD', 'I-PROD', 'B-WA', 'I-WA'] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


              precision    recall  f1-score   support

        B-EV      0.616     0.304     0.407       148
        I-EV      0.537     0.367     0.436       199
       B-LOC      0.771     0.787     0.779       437
       I-LOC      0.686     0.602     0.641       196
       B-ORG      0.659     0.452     0.537       389
       I-ORG      0.554     0.482     0.515       193
       B-PER      0.904     0.742     0.815      1148
       I-PER      0.888     0.864     0.876       603
      B-PROD      0.876     0.827     0.851       676
      I-PROD      0.741     0.576     0.648       243
        B-WA      0.333     0.036     0.065        28
        I-WA      0.250     0.064     0.102        47

   micro avg      0.805     0.679     0.737      4307
   macro avg      0.651     0.509     0.556      4307
weighted avg      0.789     0.679     0.725      4307

